In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
path_to_csv = 'data/generated/setups/patient_2/electrodes.csv'
df = pd.read_csv(path_to_csv)

In [ ]:
ele_names = df.NAME

In [ ]:
ele_names

%run solve_sphere_on_plate.py \
  --mesh data/generated/meshes/four_spheres_csf_3_mm__plain/normal.xdmf \
  --degree 1 \
  --config data/bundled/model_properties/four_spheres_csf_3_mm__Naess_Chintaluri_2017.ini \
  --grounded-plate-edge-z -0.088 \
  --electrodes data/generated/setups/patient_2/electrodes.csv \
  --name HB-3 \
  --output data/generated/tutorial/patient_2/fenics_leadfield_corrections/HB-3.ini

%run create_grid.py \
  --grid data/generated/tutorial/patient_2/sampled_leadfield_corrections/grid.npz \
  --start -0.079 \
  --end 0.079 \
  -k 9

%run sample_spherical_solution.py \
  --fill 0 \
  --sampling-radius 0.079 \
  --grid data/generated/tutorial/patient_2/sampled_leadfield_corrections/grid.npz \
  --config data/generated/tutorial/patient_2/fenics_leadfield_corrections/HB-3.ini \
  --output data/generated/tutorial/patient_2/sampled_leadfield_corrections/HB-3.npz

In [ ]:
X = [0, 0, 0, 0, 0, 0.079, -0.079]
Y = [0, 0.079, -0.079, 0, 0, 0, 0]
Z = [0, 0, 0, 0.079, -0.079, 0, 0]


df = pd.read_csv(path_to_csv)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(df.X, df.Y, df.Z)
ax.scatter(X, Y, Z)

r = 0.079
# draw sphere
u, v = np.mgrid[0:2*np.pi:50j, 0:np.pi:50j]
x = r*np.cos(u)*np.sin(v)
y = r*np.sin(u)*np.sin(v)
z = r*np.cos(v)
ax.plot_surface(x, y, z, color='b', alpha=0.2)

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

In [ ]:
from kesi.kernel.electrode import LinearlyInterpolatedLeadfieldCorrection as Electrode

In [ ]:
import glob


electrodes = [Electrode(filename)
              for filename in glob.glob("data/generated/tutorial/patient_2/sampled_leadfield_corrections/*.npz")]

In [ ]:
from kesi.common import SphericalSplineSourceKCSD


SRC_R = 10e-3

spline_nodes = [SRC_R / 3, SRC_R]
spline_polynomials = [[1],
                      [0,
                       6.75 / SRC_R,
                       -13.5 / SRC_R ** 2,
                       6.75 / SRC_R ** 3]]
model_src = SphericalSplineSourceKCSD(0, 0, 0,
                                      spline_nodes,
                                      spline_polynomials)

In [ ]:
import numpy as np

from kesi.kernel.constructor import Convolver


BRAIN_RADIUS = 0.079
ROMBERG_K = 4

_h_min = SRC_R * 2**(1 - ROMBERG_K)

#_X = _Y = np.linspace(-BRAIN_RADIUS, BRAIN_RADIUS, int(np.floor(2 * BRAIN_RADIUS / _h_min)) + 1)
#_Z = np.linspace(-0.5 * BRAIN_RADIUS, BRAIN_RADIUS, int(np.floor(1.5 * BRAIN_RADIUS / _h_min)) + 1)

_X = _Y = np.linspace(-BRAIN_RADIUS, BRAIN_RADIUS, 95)
_Z = np.linspace(-0.5 * BRAIN_RADIUS, BRAIN_RADIUS, 75)

_pot_grid = [_X, _Y, _Z]
_csd_grid = _pot_grid

convolver = Convolver(_pot_grid, _csd_grid)

for _c, _h in zip("XYZ", convolver.steps("POT")):
    assert _h >= _h_min, f"{_c}:\t{_h} < {_h_min}"
    if _h >= 2 * _h_min:
        print(f"You can reduce number of nodes of quadrature for {_c} dimension")

In [ ]:
_X.shape

In [ ]:
from kesi.kernel.constructor import ConvolverInterfaceIndexed

from scipy.integrate import romb


ROMBERG_N = 2 ** ROMBERG_K + 1
ROMBERG_WEIGHTS = romb(np.identity(ROMBERG_N)) * 2 ** -ROMBERG_K

SRC_MASK = ((convolver.SRC_Z > -0.06 + SRC_R)
            & (np.sqrt(np.square(convolver.SRC_X)
                       + np.square(convolver.SRC_Y)
                       + np.square(convolver.SRC_Z)) < BRAIN_RADIUS - SRC_R))

In [ ]:
SRC_MASK.shape

In [ ]:
convolver_interface = ConvolverInterfaceIndexed(convolver,
                                                model_src.csd,
                                                ROMBERG_WEIGHTS,
                                                SRC_MASK)

In [ ]:
from kesi.kernel import potential_basis_functions as pbf

In [ ]:
pbf_kcsd = pbf.Analytical(convolver_interface,
                          potential=model_src.potential)

In [ ]:
pbf_kesi = pbf.AnalyticalCorrectedNumerically(convolver_interface,
                                              potential=model_src.potential)

In [ ]:
from kesi.kernel.constructor import KernelConstructor, CrossKernelConstructor


kernel_constructor = KernelConstructor()

In [ ]:
CSD_MASK = np.ones(convolver.shape('CSD'),
                   dtype=bool)

kernel_constructor.crosskernel = CrossKernelConstructor(convolver_interface,
                                                        CSD_MASK)

In [ ]:
convolver.shape('CSD')

In [ ]:
convolver.CSD_GRID[0].shape

In [ ]:
def to_3D(CSD):
    return CSD.reshape(convolver.shape('CSD'))

In [ ]:
%%time
B_KCSD = kernel_constructor.potential_basis_functions_at_electrodes(electrodes,
                                                                    pbf_kcsd)

In [ ]:
%%time
KERNEL_KCSD = kernel_constructor.kernel(B_KCSD)

In [ ]:
KERNEL_KCSD.shape

In [ ]:
%%time
CROSSKERNEL_KCSD = kernel_constructor.crosskernel(B_KCSD)

In [ ]:
CROSSKERNEL_KCSD.shape

In [ ]:
del B_KCSD  # the array is large and no longer needed

In [ ]:
from kesi._verbose import _CrossKernelReconstructor as Reconstructor
from kesi._engine import _LinearKernelSolver as KernelSolver


reconstructor_kcsd = Reconstructor(KernelSolver(KERNEL_KCSD),
                                   CROSSKERNEL_KCSD)

In [ ]:
import matplotlib.pyplot as plt

from local import cbf
from local.tutorial.view import FourSpheres3mm as CoordinatePlanesVisualisation

In [ ]:
csd_grid = [_x.flatten() for _x in convolver.CSD_GRID]

In [ ]:
csd_plotter = CoordinatePlanesVisualisation(csd_grid,
                                            [0.045, 0.015, -0.02],
                                            unit_factor=1e-3,
                                            unit='$\\frac{pA}{mm^3}$',
                                            length_factor=1e3,
                                            length_unit='$mm$')

In [ ]:
%%time
EIGENVALUES_KCSD, EIGENVECTORS_KCSD = np.linalg.eigh(KERNEL_KCSD)
EIGENVALUES_KCSD, EIGENVECTORS_KCSD = EIGENVALUES_KCSD[::-1], EIGENVECTORS_KCSD[:, ::-1]

GT_CSD = to_3D(reconstructor_kcsd(EIGENVECTORS_KCSD[:, 2] * 100))

In [ ]:
csd_plotter.plot_volume(GT_CSD, 'GT CSD')

In [ ]:
import scipy.interpolate as si

from local.tutorial import SphericalForwardModel as ForwardModel

In [ ]:
%%time
fem = ForwardModel(mesh="data/generated/meshes/four_spheres_csf_3_mm__plain/fine.xdmf",
                   degree=1,
                   config="data/bundled/model_properties/four_spheres_csf_3_mm__Naess_Chintaluri_2017.ini")

In [ ]:
%%time
_csd = si.RegularGridInterpolator(csd_grid,
                                  GT_CSD,
                                  bounds_error=False,
                                  fill_value=0)
potential = fem(_csd)

del _csd  # the object is large and no longer needed

In [ ]:
potential_plotter = CoordinatePlanesVisualisation(csd_grid,
                                                  [-0.05, 0.015, -0.02],
                                                  unit='$\mu{}V$',
                                                  length_factor=1e3,
                                                  length_unit='$mm$',
                                                  cmap=cbf.PRGn)

In [ ]:
%%time
V_PLANES = []

for _X, _Y, _Z in potential_plotter.PLANES_XYZ:
    with np.nditer([np.reshape(_X, (-1, 1, 1)),
                    np.reshape(_Y, (1, -1, 1)),
                    np.reshape(_Z,  (1, 1, -1)),
                    None]) as it:
        for _x, _y, _z, _res in it:
            try:
                _res[...] = potential(_x, _y, _z)
            except RuntimeError:
                _res[...] = np.nan

        V_PLANES.append(np.ma.masked_invalid(it.operands[3]))

In [ ]:
potential_plotter.plot_planes(V_PLANES, 'POTENTIAL')

In [ ]:
GT_V = np.array([potential(_e.x, _e.y, _e.z) for _e in electrodes])

In [ ]:
del potential, fem  # these objects are large and no longer needed

In [ ]:
np.random.seed(42)
NOISE_V = np.random.normal(loc=GT_V,
                           scale=0.01 * GT_V.std())

In [ ]:
plt.stem(abs(np.matmul(GT_V, EIGENVECTORS_KCSD)))

In [ ]:
plt.stem(abs(np.matmul(GT_V, EIGENVECTORS_KCSD)) / np.sqrt(EIGENVALUES_KCSD))
plt.yscale('log')

In [ ]:
plt.stem(abs(np.matmul(NOISE_V, EIGENVECTORS_KCSD)))

In [ ]:
plt.stem(abs(np.matmul(NOISE_V, EIGENVECTORS_KCSD)) / np.sqrt(EIGENVALUES_KCSD))
plt.yscale('log')

## kCSD

As the ground truth CSD is a kCSD eigensource, kCSD is (theoretically) able to reconstruct it exactly.  But the forward model does not conform to assumptions of kCSD, which may lead to reconstruction artifacts.

In [ ]:
CSD_KCSD = to_3D(reconstructor_kcsd(NOISE_V))

In [ ]:
AMP = np.max(abs(CSD_KCSD))

In [ ]:
csd_plotter.compare_with_gt(GT_CSD,
                            CSD_KCSD,
                            'kCSD',
                            amp=AMP)

In [ ]:
np.savez_compressed("ES2_CSD_KCSD.npz", CSD=CSD_KCSD, X=convolver.CSD_X, Y=convolver.CSD_Y, Z=convolver.CSD_Z)

As you see, the reconstruction is mostly error.

In [ ]:
del CSD_KCSD  # the array is large and no longer needed

## Regularization

### Leave-one-out cross-validation

In [ ]:
plt.figure(figsize=(18, 12))
plt.plot(EIGENVALUES_KCSD)
plt.yscale('log')

In [ ]:
REGULARIZATION_PARAMETERS = np.logspace(-2, 15, 17 * 10 + 1)

In [ ]:
from kesi.common import cv

In [ ]:
%%time
CV_ERRORS_KCSD = cv(reconstructor_kcsd, NOISE_V, REGULARIZATION_PARAMETERS)

It may take more than two hours if other tasks are running.  Otherwise it may take less than a minute.

In [ ]:
regularization_parameter_kcsd = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KCSD)]

In [ ]:
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KCSD,
         color=cbf.BLUE)
plt.plot(regularization_parameter_kcsd,
         min(CV_ERRORS_KCSD),
         marker="o",
         color=cbf.BLUE)
#plt.axvspan(EIGENVALUES_KCSD.min(),
#            EIGENVALUES_KCSD.max(),
#            ls=":",
#            color=cbf.GREEN,
#            hatch=".....",
#            fill=False)
for ev in EIGENVALUES_KCSD:
    plt.axvline(ev, color=cbf.GREEN, ls=":")

# plt.axvline(EIGENVALUES_KCSD[0])
# plt.axvline(EIGENVALUES_KCSD[-1])
plt.xscale('log')
plt.xlabel('regularization parameter')
plt.yscale('log')
plt.ylabel('L2 norm of cross-validation error')

## Regularized kCSD

In [ ]:
CSD_KCSD_CV = to_3D(reconstructor_kcsd(NOISE_V, regularization_parameter_kcsd))

In [ ]:
csd_plotter.compare_with_gt(GT_CSD,
                            CSD_KCSD_CV,
                            'kCSD (regularized)')

In [ ]:
np.savez_compressed("ES2_CSD_KCSD_CV.npz", CSD=CSD_KCSD_CV, X=convolver.CSD_X, Y=convolver.CSD_Y, Z=convolver.CSD_Z)

The reconstruction still is mostly error.

In [ ]:
del CSD_KCSD_CV  # the array is large and no longer needed

In [ ]:
del CROSSKERNEL_KCSD #del reconstructor_kcsd, CROSSKERNEL_KCSD  # These objects are large and no longer needed

## Unregularized kESI

In [ ]:
%%time
B_KESI = kernel_constructor.potential_basis_functions_at_electrodes(electrodes,
                                                                    pbf_kesi)

It may take more than 20 minutes.

In [ ]:
%%time
KERNEL_KESI = kernel_constructor.kernel(B_KESI)

In [ ]:
%%time
CROSSKERNEL_KESI = kernel_constructor.crosskernel(B_KESI)

In [ ]:
KERNEL_KESI[3, :]

In [ ]:
KERNEL_KCSD[3, :]

It may take more than half a minute.

In [ ]:
del B_KESI  # the array is large and no longer needed

In [ ]:
reconstructor_kesi = Reconstructor(KernelSolver(KERNEL_KESI),
                                   CROSSKERNEL_KESI)

In [ ]:
CSD_KESI = to_3D(reconstructor_kesi(NOISE_V))

In [ ]:
csd_plotter.compare_with_gt(GT_CSD,
                            CSD_KESI,
                            'kESI')

In [ ]:
np.savez_compressed("ES2_CSD_KESI.npz", CSD=CSD_KESI, X=convolver.CSD_X, Y=convolver.CSD_Y, Z=convolver.CSD_Z)

As you see, the unregularized kESI reconstruction somehow resembles the ground truth, but still is mostly error.

In [ ]:
del CSD_KESI  # the array is large and no longer needed

## Regularized kESI

In [ ]:
EIGENVALUES_KESI, EIGENVECTORS_KESI = np.linalg.eigh(KERNEL_KESI)
EIGENVALUES_KESI, EIGENVECTORS_KESI = EIGENVALUES_KESI[::-1], EIGENVECTORS_KESI[:, ::-1]

In [ ]:
plt.figure(figsize=(18, 12))
plt.plot(EIGENVALUES_KCSD, label='kCSD', ls='-')
plt.plot(EIGENVALUES_KESI, label='kESI', ls=':')
plt.yscale('log')
plt.legend(loc='best')

In [ ]:
from kesi.common import cv

In [ ]:
# REGULARIZATION_PARAMETERS = np.logspace(-2, 15, 17 * 10 + 1)

In [ ]:
%%time
CV_ERRORS_KESI = cv(reconstructor_kesi, NOISE_V, REGULARIZATION_PARAMETERS)

It may take more than 2 hours if other tasks are running.  Otherwise it may take less than a minute.

In [ ]:
regularization_parameter_kesi = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KESI)]

In [ ]:
plt.plot(REGULARIZATION_PARAMETERS,
        CV_ERRORS_KCSD,
        color=cbf.BLUE,
        label='kCSD')
plt.plot(regularization_parameter_kcsd,
        min(CV_ERRORS_KCSD),
        marker="o",
        color=cbf.BLUE)
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KESI,
         color=cbf.VERMILION,
         label='kESI')
plt.plot(regularization_parameter_kesi,
         min(CV_ERRORS_KESI),
         marker="o",
         color=cbf.VERMILION)
# plt.axvspan(min(EIGENVALUES_KCSD.min(), EIGENVALUES_KESI.min()),
#             max(EIGENVALUES_KESI.max(), EIGENVALUES_KESI.max()),
#             ls=":",
#             color=cbf.GREEN,
#             hatch=".....",
#             fill=False)
for ev in EIGENVALUES_KESI:
    plt.axvline(ev, color=cbf.GREEN, ls=":")
plt.xscale('log')
plt.xlabel('regularization parameter')
plt.yscale('log')
plt.ylabel('L2 norm of cross-validation error')
plt.legend(loc='best')

In [ ]:
CSD_KESI_CV = to_3D(reconstructor_kesi(NOISE_V, regularization_parameter_kesi))

In [ ]:
csd_plotter.compare_with_gt(GT_CSD,
                            CSD_KESI_CV,
                            'kESI (regularized)')

In [ ]:
np.savez_compressed("ES2_CSD_KESI_CV.npz", CSD=CSD_KESI_CV, X=convolver.CSD_X, Y=convolver.CSD_Y, Z=convolver.CSD_Z)

In [ ]:
plt.stem(abs(np.matmul(GT_V, EIGENVECTORS_KESI)))

In [ ]:
plt.stem(abs(np.matmul(GT_V, EIGENVECTORS_KESI)) / np.sqrt(EIGENVALUES_KESI))
plt.yscale('log')

In [ ]:
plt.stem(abs(np.matmul(NOISE_V, EIGENVECTORS_KESI)))

In [ ]:
plt.stem(abs(np.matmul(NOISE_V, EIGENVECTORS_KESI)) / np.sqrt(EIGENVALUES_KESI))
plt.yscale('log')

The kESI reconstruction definitely benefitted from regularization.

In [ ]:
del CSD_KESI_CV  # the array is large and no longer needed

#### Analysis of epileptic data

In [ ]:
from scipy.signal import filtfilt, butter
Fs=1000
npz_file = np.load('epi_data_to_vis_clinic_Pat2.npz')
seeg = npz_file['eeg']
print('seeg', seeg.shape)
b, a = butter(3, [1/(Fs/2), 400/(Fs/2)], btype='bandpass')
seeg= filtfilt(b, a, seeg)

In [ ]:
from scipy.signal import filtfilt, butter
Fs=1000
seizures = np.load('lfp_napad_clinic_pat2.npy')
print('seizures', seizures.shape)
b, a = butter(3, [1/(Fs/2), 400/(Fs/2)], btype='bandpass')
seizures = filtfilt(b, a, seizures)

In [ ]:
SEEG_PART = seizures[:, 120000:150000:10]*1e3

In [ ]:
SEEG_PART.shape

In [ ]:
(1. / np.sqrt(EIGENVALUES_KESI.reshape(-1, 1)))

In [ ]:
_ESS = np.matmul(1. / np.sqrt(EIGENVALUES_KESI.reshape(-1, 1))
                                 * EIGENVECTORS_KESI.T,
                                 SEEG_PART)
for i, ES in enumerate(_ESS):
    plt.figure(figsize=(16, 4))
    plt.title(i)
    plt.plot(ES)

_ESS = abs(_ESS)
_ESS /= _ESS.sum(axis=0).reshape(1, -1)
max_es_at = {i: ES.argmax() for i, ES in enumerate(_ESS)}

del _ESS

In [ ]:
plt.figure(figsize=(16, 4))
plt.plot(SEEG_PART[0, :])

In [ ]:
from scipy.signal import spectrogram
plt.figure()
Fs=1000
f, t, Sxx = spectrogram(seizures[15,:].T, Fs, nperseg=1000)
plt.pcolormesh(t, f, Sxx)
plt.ylabel('Frequency (Hz)'), plt.xlabel('Time (s)'), plt.ylim(0, 50)

In [ ]:
plt.figure()
Fs=1000
f, t, Sxx = spectrogram(SEEG_PART[15,:].T, Fs, nperseg=100)
plt.pcolormesh(t, f, Sxx)
plt.ylabel('Frequency (Hz)'), plt.xlabel('Time (s)'), plt.ylim(0, 150)

In [ ]:
plt.figure()
Fs=1000
f, t, Sxx = spectrogram(seeg[15,:].T, Fs, nperseg=1000)
plt.pcolormesh(t, f, Sxx)
plt.ylabel('Frequency (Hz)'), plt.xlabel('Time (s)'), plt.ylim(0, 50)

#### kESI with real data

In [ ]:
SEEG_TIMEPOINT = SEEG_PART[:, 2200]

In [ ]:
plt.stem(abs(np.matmul(SEEG_TIMEPOINT, EIGENVECTORS_KESI)))

In [ ]:
plt.stem(abs(np.matmul(SEEG_TIMEPOINT, EIGENVECTORS_KESI)) / np.sqrt(EIGENVALUES_KESI))
plt.yscale('log')

In [ ]:
plt.stem(abs(np.matmul(SEEG_TIMEPOINT, EIGENVECTORS_KCSD)))

In [ ]:
plt.stem(abs(np.matmul(SEEG_TIMEPOINT, EIGENVECTORS_KCSD)) / np.sqrt(EIGENVALUES_KCSD))
plt.yscale('log')

In [ ]:
%%time
CSD_timeseries_kESI = reconstructor_kesi(SEEG_PART)

In [ ]:
CSD_timeseries_kESI.shape

In [ ]:
csd_plotter.plot_volume(to_3D(CSD_timeseries_kESI[:, 2200]), 'kESI without regularization')

In [ ]:
%%time
CV_ERRORS_KESI_SEEG = cv(reconstructor_kesi, SEEG_PART[:, 1000:3000], REGULARIZATION_PARAMETERS)

In [ ]:
%%time
CV_ERRORS_KESI_SEEG_TIMEPOINT = cv(reconstructor_kesi, SEEG_TIMEPOINT, REGULARIZATION_PARAMETERS)

In [ ]:
regularization_parameter_kesi_seeg = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KESI_SEEG)]

In [ ]:
regularization_parameter_kesi_seeg_timepoint = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KESI_SEEG_TIMEPOINT)]

In [ ]:
regularization_parameter_kesi_seeg

In [ ]:
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KESI_SEEG,
         color=cbf.VERMILION,
         label='kESI timespan')
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KESI_SEEG_TIMEPOINT,
         color=cbf.ORANGE,
         label='kESI timepoint 2200')
plt.plot(regularization_parameter_kesi_seeg,
         min(CV_ERRORS_KESI_SEEG),
         marker="o",
         color=cbf.VERMILION)
plt.plot(regularization_parameter_kesi_seeg_timepoint,
         min(CV_ERRORS_KESI_SEEG_TIMEPOINT),
         marker="o",
         color=cbf.ORANGE)

# plt.axvspan(min(EIGENVALUES_KCSD.min(), EIGENVALUES_KESI.min()),
#             max(EIGENVALUES_KESI.max(), EIGENVALUES_KESI.max()),
#             ls=":",
#             color=cbf.GREEN,
#             hatch=".....",
#             fill=False)
for ev in EIGENVALUES_KESI:
    plt.axvline(ev, color=cbf.GREEN, ls=":")
# plt.axvline(EIGENVALUES_KESI[0])
# plt.axvline(EIGENVALUES_KESI[-1])
plt.xscale('log')
plt.xlabel('regularization parameter')
plt.yscale('log')
plt.ylabel('L2 norm of cross-validation error')
plt.legend(loc='best')

In [ ]:
%%time
CSD_TIMESERIES_KESI_CV = reconstructor_kesi(SEEG_PART[:, :], regularization_parameter_kesi_seeg)

In [ ]:
%%time
CSD_TIMESERIES_KESI_CV_TIMEPOINT = reconstructor_kesi(SEEG_TIMEPOINT, regularization_parameter_kesi_seeg_timepoint)

In [ ]:
convolver.CSD_X.flatten()

In [ ]:
a,b,c = np.meshgrid(convolver.CSD_X.flatten(), convolver.CSD_Y.flatten(), convolver.CSD_Z.flatten())

In [ ]:
a.shape

In [ ]:
np.savez_compressed('CSD_TIMESERIES_KESI_CV', CSD=CSD_TIMESERIES_KESI_CV, X=convolver.CSD_X, Y=convolver.CSD_Y, Z=convolver.CSD_Z )

In [ ]:
csd_plotter.plot_volume(to_3D(CSD_TIMESERIES_KESI_CV[:, 2200]), 'kESI with regularization (timespan)')

In [ ]:
csd_plotter.plot_volume(to_3D(CSD_TIMESERIES_KESI_CV_TIMEPOINT), 'kESI with regularization (timepoint)')

#### kCSD with real data

In [ ]:
%time
CSD_timeseries_kCSD = reconstructor_kcsd(SEEG_PART)

In [ ]:
CSD_timeseries_kCSD.shape

In [ ]:
csd_plotter.plot_volume(to_3D(CSD_timeseries_kCSD[:, 2200]), 'kCSD without regularization')

In [ ]:
%%time
CV_ERRORS_KCSD_SEEG = cv(reconstructor_kcsd, SEEG_PART[:, 1000:3000], REGULARIZATION_PARAMETERS)

In [ ]:
%%time
CV_ERRORS_KCSD_SEEG_TIMEPOINT = cv(reconstructor_kcsd, SEEG_TIMEPOINT, REGULARIZATION_PARAMETERS)

In [ ]:
regularization_parameter_kcsd_seeg = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KCSD_SEEG)]

In [ ]:
regularization_parameter_kcsd_seeg_timepoint = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KCSD_SEEG_TIMEPOINT)]

In [ ]:
regularization_parameter_kcsd_seeg

In [ ]:
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KCSD_SEEG,
         color=cbf.BLUE)
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KCSD_SEEG_TIMEPOINT,
         color=cbf.SKY_BLUE)
plt.plot(regularization_parameter_kcsd_seeg,
         min(CV_ERRORS_KCSD_SEEG),
         marker="o",
         color=cbf.BLUE)
plt.plot(regularization_parameter_kcsd_seeg_timepoint,
         min(CV_ERRORS_KCSD_SEEG_TIMEPOINT),
         marker="o",
         color=cbf.SKY_BLUE)
plt.axvspan(EIGENVALUES_KCSD.min(),
            EIGENVALUES_KCSD.max(),
            ls=":",
            color=cbf.GREEN,
            hatch=".....",
            fill=False)
plt.xscale('log')
plt.xlabel('regularization parameter')
plt.yscale('log')
plt.ylabel('L2 norm of cross-validation error')

In [ ]:
plt.stem(abs(np.matmul(SEEG_TIMEPOINT, EIGENVECTORS_KCSD)) / np.sqrt(EIGENVALUES_KCSD))
plt.plot(abs(np.matmul(SEEG_TIMEPOINT, EIGENVECTORS_KCSD)) / np.sqrt(EIGENVALUES_KCSD + regularization_parameter_kcsd_seeg),
         color=cbf.ORANGE)

plt.yscale('log')

In [ ]:
%%time
CSD_TIMESERIES_KCSD_CV = reconstructor_kcsd(SEEG_PART[:, 2200], regularization_parameter_kcsd_seeg)

In [ ]:
%%time
CSD_TIMESERIES_KCSD_CV_TIMEPOINT = reconstructor_kcsd(SEEG_TIMEPOINT, regularization_parameter_kcsd_seeg_timepoint)

In [ ]:
CSD_TIMESERIES_KCSD_CV.shape

In [ ]:
csd_plotter.plot_volume(to_3D(CSD_TIMESERIES_KCSD_CV), 'kCSD with regularization (timespan)')

In [ ]:
csd_plotter.plot_volume(to_3D(CSD_TIMESERIES_KCSD_CV_TIMEPOINT), 'kCSD with regularization (timespan)')

In [ ]:
del CSD_timeseries_kESI

In [ ]:
del CSD_timeseries_kCSD

In [ ]:
T = 3000
T_START = 125000
T_END = 150000
TIME = np.linspace(T_START, T_END, T)

### ICA computations

In [ ]:
import numpy as np
import pylab as py
from sklearn.decomposition import FastICA
from scipy.signal import spectrogram
import matplotlib.colors as colors

In [ ]:
def comp_ica(csd, comps=4, i_comp=0):
    nmf = FastICA(n_components=comps, random_state=0)
    nmf_1, A = nmf.fit_transform(csd), nmf.mixing_
    # comp = np.dot(nmf_1[:,i_comp].reshape(len(nmf_1),1),A1.T)
    return nmf_1, A

In [ ]:
#filename = 'epi_data_to_vis_clinic_Pat2_corrected.npz'
#npz_file = np.load(filename)
#csd=npz_file['csd'][:,:]*1e3
#eeg=seeg[:,:]*1e3
ch_list = np.loadtxt('channels_for_clinic_pat2.txt', dtype=np.str_)

In [ ]:
vect1, vect2 = comp_ica(CSD_TIMESERIES_KESI_CV[:,:].T)
#np.save('ica_patient2', vect2)

In [ ]:
py.figure()
Fs=1000
py.suptitle(filename)
for i in range(4):
    py.subplot(4,1,i+1)
    py.title('ica'+str(i))
    f, t, Sxx = spectrogram(vect1[:,i], Fs, nperseg=Fs)
    py.pcolormesh(t, f, Sxx, vmax=1e-6)
    py.ylabel('Frequency (Hz)'), py.xlabel('Time (s)'), py.ylim(0,50)

In [ ]:
ES = 9
TIMEPOINT_ES = SEEG_PART[:, max_es_at[ES]]

CV_ERRORS_KCSD_EV = cv(reconstructor_kcsd, TIMEPOINT_ES, REGULARIZATION_PARAMETERS)
regularization_parameter_kcsd_es = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KCSD_EV)]

CV_ERRORS_KESI_EV = cv(reconstructor_kesi, TIMEPOINT_ES, REGULARIZATION_PARAMETERS)
regularization_parameter_kesi_es = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KESI_EV)]

In [ ]:
plt.title(f"ES #{ES}")
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KCSD_EV,
         color=cbf.BLUE)
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KESI_EV,
         color=cbf.VERMILION)
plt.plot(regularization_parameter_kcsd_es,
         min(CV_ERRORS_KCSD_EV),
         marker="o",
         color=cbf.BLUE,
         label="kCSD")
plt.plot(regularization_parameter_kesi_es,
         min(CV_ERRORS_KESI_EV),
         marker="o",
         color=cbf.VERMILION,
         label="kESI")
plt.axvspan(EIGENVALUES_KCSD.min(),
            EIGENVALUES_KCSD.max(),
            ls=":",
            color=cbf.GREEN,
            hatch=".....",
            fill=False)
plt.xscale('log')
plt.xlabel('regularization parameter')
plt.yscale('log')
plt.ylabel('L2 norm of cross-validation error')
plt.legend(loc="best");

In [ ]:
csd_plotter.plot_volume(to_3D(reconstructor_kesi(TIMEPOINT_ES)), 'kESI without regularization')

In [ ]:
csd_plotter.plot_volume(to_3D(reconstructor_kcsd(TIMEPOINT_ES)), 'kCSD without regularization')

In [ ]:
csd_plotter.plot_volume(to_3D(reconstructor_kesi(TIMEPOINT_ES, regularization_parameter_kesi_es)), 'kESI with regularization')

In [ ]:
csd_plotter.plot_volume(to_3D(reconstructor_kcsd(TIMEPOINT_ES, regularization_parameter_kcsd_es)),
                        'kCSD with regularization')

In [ ]:
_CSD = to_3D(reconstructor_kcsd(TIMEPOINT_ES, regularization_parameter_kcsd_es))
_CSD[0,0,0] = 100 * abs(_CSD).max()
csd_plotter.plot_volume(_CSD,
                        'kCSD with regularization')

In [ ]:
i_max_L2_diff = None
max_L2_diff = 0
for i, _TIMEPOINT in enumerate(SEEG_PART.T):
    _CV_ERRORS_KCSD = cv(reconstructor_kcsd, _TIMEPOINT, REGULARIZATION_PARAMETERS)
    _regularization_parameter_kcsd = REGULARIZATION_PARAMETERS[np.argmin(_CV_ERRORS_KCSD)]

    _CV_ERRORS_KESI = cv(reconstructor_kesi, _TIMEPOINT, REGULARIZATION_PARAMETERS)
    _regularization_parameter_kesi = REGULARIZATION_PARAMETERS[np.argmin(_CV_ERRORS_KESI)]

    _KCSD = reconstructor_kcsd(_TIMEPOINT, _regularization_parameter_kcsd)
    _KESI = reconstructor_kesi(_TIMEPOINT, _regularization_parameter_kesi)
    _amp = 0.5 * (abs(_KCSD).max() + abs(_KESI).max())
    _DIFF = to_3D(_KCSD - _KESI)

    _L2 = np.sqrt(np.square(_DIFF).mean()) / _amp
    if _L2 > max_L2_diff:
        max_L2_diff = _L2
        i_max_L2_diff = i

In [ ]:
print(i_max_L2_diff, max_L2_diff)

In [ ]:
TIMEPOINT_BF = SEEG_PART[:, i_max_L2_diff]

CV_ERRORS_KCSD_BF = cv(reconstructor_kcsd, TIMEPOINT_BF, REGULARIZATION_PARAMETERS)
regularization_parameter_kcsd_bf = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KCSD_BF)]

CV_ERRORS_KESI_BF = cv(reconstructor_kesi, TIMEPOINT_BF, REGULARIZATION_PARAMETERS)
regularization_parameter_kesi_bf = REGULARIZATION_PARAMETERS[np.argmin(CV_ERRORS_KESI_BF)]

In [ ]:
plt.title(f"BF: timepoint #{i_max_L2_diff}")
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KCSD_BF,
         color=cbf.BLUE)
plt.plot(REGULARIZATION_PARAMETERS,
         CV_ERRORS_KESI_BF,
         color=cbf.VERMILION)
plt.plot(regularization_parameter_kcsd_bf,
         min(CV_ERRORS_KCSD_BF),
         marker="o",
         color=cbf.BLUE,
         label="kCSD")
plt.plot(regularization_parameter_kesi_bf,
         min(CV_ERRORS_KESI_BF),
         marker="o",
         color=cbf.VERMILION,
         label="kESI")
plt.axvspan(EIGENVALUES_KCSD.min(),
            EIGENVALUES_KCSD.max(),
            ls=":",
            color=cbf.GREEN,
            hatch=".....",
            fill=False)
plt.xscale('log')
plt.xlabel('regularization parameter')
plt.yscale('log')
plt.ylabel('L2 norm of cross-validation error')
plt.legend(loc="best");

In [ ]:
csd_plotter.plot_volume(to_3D(reconstructor_kesi(TIMEPOINT_BF)), 'kESI without regularization')

In [ ]:
csd_plotter.plot_volume(to_3D(reconstructor_kcsd(TIMEPOINT_BF)), 'kCSD without regularization')

In [ ]:
csd_plotter.plot_volume(to_3D(reconstructor_kesi(TIMEPOINT_BF, regularization_parameter_kesi_bf)), 'kESI with regularization')

In [ ]:
csd_plotter.plot_volume(to_3D(reconstructor_kcsd(TIMEPOINT_BF, regularization_parameter_kcsd_bf)),
                        'kCSD with regularization')

In [ ]:
_A = np.log10(abs(np.matmul(EIGENVECTORS_KCSD.T, EIGENVECTORS_KESI)))
_amp = abs(_A).max()

plt.imshow(_A, vmin=-_amp, vmax=_amp, cmap=cbf.bwr)
plt.colorbar()

In [ ]:
plt.stem(abs(np.matmul(NOISE_V, EIGENVECTORS_KCSD)) / np.sqrt(EIGENVALUES_KCSD))
plt.yscale('log')

In [ ]:
plt.stem(abs(np.matmul(GT_V, EIGENVECTORS_KESI)))
plt.yscale('log')

In [ ]:
plt.stem(abs(np.matmul(GT_V, EIGENVECTORS_KCSD)))
plt.yscale('log')

In [ ]:
plt.stem(abs(np.matmul(NOISE_V, EIGENVECTORS_KESI)) / np.sqrt(EIGENVALUES_KESI))
plt.yscale('log')